In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
from langchain_upstage import UpstageDocumentParseLoader
from langchain_teddynote import logging

In [10]:

# API KEY 정보로드
load_dotenv()

# LangSmith 추적을 설정합니다. https://smith.langchain.com


# 프로젝트 이름을 입력합니다.
logging.langsmith("CH07-DocumentLoader")


LangSmith 추적을 시작합니다.
[프로젝트명]
CH07-DocumentLoader


In [ ]:

# 파일 경로
file_path = ".\data\중소벤처기업 지원사업_샘플.pdf"

layzer = UpstageDocumentParseLoader(
            file_path,
            split='page',
            output_format='html',
           )

docs = layzer.load()

for doc in docs[:3]:
    print(doc)
    

page_content='<h1 id='0' style='font-size:16px'>1-1</h1> <br><h1 id='1' style='font-size:18px'>정책자금지원 공통사항</h1> <br><h1 id='2' style='font-size:22px'>节目的</h1> <h1 id='3' style='font-size:22px'>정책자금 지원시 공통적으로 적용되는 사항</h1> <h1 id='4' style='font-size:20px'>사업개요</h1> <p id='5' data-category='paragraph' style='font-size:18px'>혁신창업사업화자금 등 6개 자금 및 이차보전 지원 시 융자대상 및<br>절차, 융자제한기업, 융자시기 등 공통적으로 적용</p> <h1 id='6' style='font-size:20px'>지원대상</h1> <p id='7' data-category='paragraph' style='font-size:20px'>성장 잠재력은 높지만 정보 비대칭 및 담보 위주 관행으로 민간 금융<br>기관에서 자금 조달이 어려운 우수 중소벤처기업</p> <br><p id='8' data-category='paragraph' style='font-size:20px'>고용창출, 수출·매출 증대 등 성과창출 기업, 시설투자 기업,<br>혁신성장분야 등 중점지원분야 영위기업에 정책자금 우선 지원</p> <p id='9' data-category='list' style='font-size:14px'>* 중점지원분야 : 혁신성장분야, 초격차·신산업 분야, 지역주력산업, 뿌리산업, 소재·<br>부품·장비 산업<br>* 중점지원분야 참고자료는 중소벤처기업진흥공단 누리집(www.kosmes.or.kr) 공지</p> <h1 id='10' style='font-size:16px'>지원이 안되는 기업</h1> <br><p id='11' data-category='list' style='font-size:14px'>▶ 중소기업 정책